In [3]:
import numpy as np
import plotly.graph_objects as go

In [63]:
class edgecnt:
    PE, CE, OE, EE, ME = 0, 0, 0, [0, 0, 0, 0], [0, 0, 0, 0]
    
ppos_req = ["overall", "noun", "verb", "properNoun", "adjective", "adverb", "determiner", "numeral", "pronoun", "preposition"]

def getRaw(lang):
    metrics = {}
    print("Language: " + lang)
    path = "Results/RemLang/Analysis/" + lang + "/POS_RAW.txt"
    inp_file = open(path, "r")
    inp = inp_file.read().splitlines()
    it = 0
    
    while(it<len(inp)):
        temp = edgecnt()
        pos = inp[it]
        temp.OE = int(inp[it+1])
        temp.CE = int(inp[it+2])
        temp.EE = inp[it+3].split()
        temp.ME = inp[it+4].split()
        it+=5
        
        for i in range(4):
            temp.EE[i] = int(temp.EE[i])
            temp.ME[i] = int(temp.ME[i])
        temp.PE = sum(temp.EE)
        temp.PE += temp.CE
        metrics.update({pos : temp})
        
    return metrics
    
def compute(metrics):
    bvP, bvR, ipnoco, ionpcd = [], [], [], []
    for i in range(5):
        ipnoco.append([])
        ionpcd.append([])
        
    for pos in ppos_req:
        m = metrics[pos]
        bvp = m.CE * 100.00/ max(1, (m.CE + m.EE[3]))
        bvP.append(bvp)
        bvr = m.CE * 100.00 / max(1, (m.CE + m.ME[3]))
        bvR.append(bvr)
        
        sumipnoco, sumionpcd = 0, 0
        for i in range(1, 5):
            sumipnoco += m.EE[i-1]*100.00/max(1, m.PE)
            sumionpcd += m.ME[i-1]*100.00/max(1, m.OE)
            ipnoco[i].append(m.EE[i-1]*100.00/max(1, m.PE))
            ionpcd[i].append(m.ME[i-1]*100.00/max(1, m.OE))
        ipnoco[0].append(100 - sumipnoco)
        ionpcd[0].append(100 - sumionpcd)
        
    return bvP, bvR, ipnoco, ionpcd

def printer(bvP, bvR, ipnoco, ionpcd):
    i = 0
    for pos in ppos_req:
        print("Part of Speech: " + pos)
        print("Precision when both vertices common in Predicted and Original: " + str(bvP[i]))
        print("Recall when both vertices commin in Original and Data: " + str(bvR[i]) + "\n")
        print("Common vertices legend: 0 = No vertex, 1 = lang1, 2 = lang2, 3 = both vertices")
        print("In Predicted, Not in Original, Classified by Original, as percentage of Predicted (Precision')")
        for j in range(1, 5):
            print(str(j-1) + " - " + str(ipnoco[j][i]))
        print("In Original, Not in Predicted, Classified by Data, as percentage of Original (Recall)")
        for j in range(1, 5):
            print(str(j-1) + " - " + str(ionpcd[j][i]))
        
        print("")
        i+=1

def prefixSum(arr):
    ps = []
    for i in range(len(arr)):
        ps.append([])
        for j in range(len(arr[i])):
            ps[i].append(arr[i][j])
            if(i>0):
                ps[i][j] += ps[i-1][j]
    return ps

def plot4in1(bvP, bvR, ipnoco, ionpcd):
    fig = go.Figure()
    fig.add_trace(go.Bar(name="Both Vertex Precision", x=ppos_req, y=bvP, base=0, offset=-0.4, width=0.2))
    fig.add_trace(go.Bar(name="Both Vertex Recall", x=ppos_req, y=bvR, base=0, offset=-0.2, width=0.2))
    
    PSipnoco = prefixSum(ipnoco)
    PSionpcd = prefixSum(ionpcd)
    
    for i in range(4, 0, -1):
        name = "iPnOcO" + str(i-1)
        fig.add_trace(go.Bar(name=name, x=ppos_req, y=PSipnoco[i], base=0, offset=0, width=0.2))
    fig.add_trace(go.Bar(name="Overall Precision", x=ppos_req, y=PSipnoco[0], base=0, offset=0, width=0.2))
    
    for i in range(4, 0, -1):
        name = "iOnPcD" + str(i-1)
        fig.add_trace(go.Bar(name=name, x=ppos_req, y=PSionpcd[i], base=0, offset=0.2, width=0.2))
    fig.add_trace(go.Bar(name="Overall Recall", x=ppos_req, y=PSionpcd[0], base=0, offset=0.2, width=0.2))
    
    
    fig.update_layout(barmode='group', width=1000, height=800, xaxis_tickangle=45)
    fig.show()
    
def plot2in2(bvP, bvR, ipnoco, ionpcd):
    fig = go.Figure()
    fig2 = go.Figure()
    fig.add_trace(go.Bar(name="Both Vertex Precision", x=ppos_req, y=bvP, base=0, offset=-0.4, width=0.4))
    fig.add_trace(go.Bar(name="Both Vertex Recall", x=ppos_req, y=bvR, base=0, offset=0, width=0.4))
    
    PSipnoco = prefixSum(ipnoco)
    PSionpcd = prefixSum(ionpcd)
    
    for i in range(4, 0, -1):
        name = "iPnOcO" + str(i-1)
        fig2.add_trace(go.Bar(name=name, x=ppos_req, y=PSipnoco[i], base=0, offset=-0.4, width=0.4))
    fig2.add_trace(go.Bar(name="Overall Precision", x=ppos_req, y=PSipnoco[0], base=0, offset=-0.4, width=0.4))
    
    for i in range(4, 0, -1):
        name = "iOnPcD" + str(i-1)
        fig2.add_trace(go.Bar(name=name, x=ppos_req, y=PSionpcd[i], base=0, offset=0, width=0.4))
    fig2.add_trace(go.Bar(name="Overall Recall", x=ppos_req, y=PSionpcd[0], base=0, offset=0, width=0.4))
    
    
    fig.update_layout(barmode='group', width=1000, height=800, xaxis_tickangle=45)
    fig.show()
    fig2.update_layout(barmode='group', width=1000, height=800, xaxis_tickangle=45)
    fig2.show()

In [64]:
metrics = getRaw("en-es")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: en-es


In [66]:
metrics = getRaw("en-ca")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: en-ca


In [67]:
metrics = getRaw("fr-es")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: fr-es


In [68]:
metrics = getRaw("fr-ca")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: fr-ca


In [69]:
metrics = getRaw("eo-fr")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: eo-fr


In [70]:
metrics = getRaw("eo-ca")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: eo-ca


In [71]:
metrics = getRaw("eo-en")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: eo-en


In [72]:
metrics = getRaw("eo-es")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: eo-es


In [73]:
metrics = getRaw("oc-ca")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: oc-ca


In [74]:
metrics = getRaw("oc-es")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: oc-es


In [75]:
metrics = getRaw("oc-fr")
bvP, bvR, ipnoco, ionpcd = compute(metrics)
plot2in2(bvP, bvR, ipnoco, ionpcd)

Language: oc-fr
